Copyright © 2020, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Delete Model Content Logs

When you publish a model to a destination, there are log and SAS code files that are generated within the contents of a model object. 
This example enables you to delete the files for a specific model, all models within a project, or all models within the common model repository. The model content is only deleted for the following file types: ScoreCodeGen{}.sas or ScoreCodeGen{}.log. The most recent revision of each file type is not deleted.

In [ ]:
import requests
import json
import getpass
from tqdm import tqdm

### Generate Authorization Token
Specify values for the `host`, `username`, and `password` parameters, and then pass an API POST request to return an authentication token. When successful the authentication response should return an HTTP status code of 200.

In [ ]:
host = 'http://myserver.com'
username = 'myUsername'
password = 'myPassword'
#password = getpass.getpass()

In [ ]:
authURI = '/SASLogon/oauth/token'

headersAuth = {
    "accept":"application/json",
    "content-type":"application/x-www-form-urlencoded",
    "Authorization":"Basic c2FzLmVjOg=="
}

authBody = f'grant_type=password&username={username}&password={password}'
authResponse = requests.post(host + authURI, data=authBody, headers=headersAuth)
print(authResponse.status_code)

In [ ]:
authToken = authResponse.json()['access_token']
print(authToken)

In [ ]:
headers = {'Authorization': f'Bearer {authToken}'}
url = f"{host}/modelRepository/repositories?filter=eq(name,'Public')"
repoList = requests.get(url, headers=headers)
repoJSON = repoList.json()['items'][0]
repoID = repoJSON['id']
print(f'Repository ID: {repoID}')
repoFID = repoJSON['folderId']
print(f'Repository Folder ID: {repoFID}')

### API Functions

In [ ]:
def inputWhileLoop(inputPrompt, validInputList):
    validInput = []
    inputAttempts = 0
    # Three input attempts are allowed before an exception is raised to close the while loop.
    while validInput not in validInputList:
        try:
            validInput = int(input(inputPrompt))
        except ValueError:
            validInput = []
        if validInput not in validInputList:
            inputAttempts += 1
            if inputAttempts >= 3:
                raise ValueError('Too many invalid input attempts have been made.')
            else:
                print('Please specify a valid option from the list.')
    return validInput - 1

In [ ]:
def findProjectUUID(host, authToken):
    isUUID = inputWhileLoop('Identify project by:\n1. Name \n2. UUID\n', [1, 2])
    if isUUID:
        projectUUID = input('Enter project UUID: ')
        return [projectUUID]
    else:
        projectName = input('Enter project name: ')
        headers = {
            'Origin': host,
            'Authorization': f'Bearer {authToken}'}
        requestUrl = f'{host}/modelRepository/projects?limit=100000'
        projectGET = requests.get(requestUrl, headers=headers)
        
        projectUUID = [x['id'] for x in projectGET.json()['items'] if x['name']==projectName]
        if not projectUUID:
            raise FileNotFoundError(f'A project with the name "{projectName}" could not be found.')
        if type(projectUUID) is list:
            return projectUUID
        else:
            return [projectUUID]

In [ ]:
def findModelUUID(host, authToken, projectUUID=None):
    isUUID = inputWhileLoop('Identify model by:\n1. Name \n2. UUID\n', [1, 2])
    modelUUID = []
    if isUUID:
        modelUUID = input('Enter model UUID: ')
        return [modelUUID]
    else:
        modelName = input('Enter model name: ')
        headers = {
            'Origin': host,
            'Authorization': f'Bearer {authToken}'}
        requestUrl = f'{host}/modelRepository/models?limit=100000'
        modelGET = requests.get(requestUrl, headers=headers)
        
        for item in modelGET.json()['items']:
            try:
                if (item['projectId']==projectUUID and item['name']==modelName):
                    modelUUID.append(item['id'])
            except KeyError:
                if projectUUID is None and item['name']==modelName:
                    modelUUID.append(item['id'])

        if not modelUUID:
            raise FileNotFoundError(f'A model with the name "{modelName}" could not be found.')
        if type(modelUUID) is list:
            return modelUUID
        else:
            return [modelUUID]

In [ ]:
def getModelContents(host, authToken, modelUUID):
    headers = {
        'Origin': host,
        'Authorization': f'Bearer {authToken}'}
    requestUrl = f'{host}/modelRepository/models/{modelUUID}/contents?limit=100000'
    contentsGET = requests.get(requestUrl, headers=headers)
    contentsNamesModified = [(x['id'], x['modifiedTimeStamp']) for x in contentsGET.json()['items'] if x['name'].startswith('ScoreCodeGen')]
    def sortTime(value):
        return value[1]
    contentsNamesModified.sort(key=sortTime, reverse=True)
    contentsUUID = [x[0] for x in contentsNamesModified[2:]]
    if type(contentsUUID) is list:
        return contentsUUID
    else:
        return [contentsUUID]

In [ ]:
def deleteModelContents(host, authToken, modelUUID, uuid):
    headers = {
        'Origin': host,
        'Authorization': f'Bearer {authToken}'}
    requestUrl = f'{host}/modelRepository/models/{modelUUID}/contents/{uuid}'
    contentDELETE = requests.delete(requestUrl, headers=headers)
    return contentDELETE

### Specify Content to Delete
Specify whether to delete generated model logs and SAS files from a model, from all models within a project, or all models within the common model repository. The model content is only deleted for the following file types: ScoreCodeGen{}.sas or ScoreCodeGen{}.log. The most recent revision of each file type is not deleted.

In [ ]:
removeType = inputWhileLoop('Specify one of the following options from which to delete the generated model logs and SAS code files: \n1. Model \n2. Project \n3. Common Model Repository\n',
                            [1, 2, 3])
# 1. Model
# Delete contents from a model using a single model UUID or a single model name.
# If a single model name, search for all versions and project locations.
deleted = 0
if removeType == 0:
    projectUUID = findProjectUUID(host, authToken)
    # if multiple project versions exist, check for the model name in each
    for pUUID in projectUUID:
        modelUUID = findModelUUID(host, authToken, pUUID)
        # if multiple model versions exist, check for/delete contents in each
        for mUUID in modelUUID:
            contentsUUID = getModelContents(host, authToken, mUUID)
            for cUUID in contentsUUID:
                code = deleteModelContents(host, authToken, mUUID, cUUID)
                if code.status_code == 204:
                    deleted += 1
    print(f'Deleted {str(deleted)} old files.')

# 2. Project
# Delete contents from models using a single project UUID or a single project name.
# If for a single project name, search through all versions of project.
elif removeType == 1:
    projectUUID = findProjectUUID(host, authToken)
    # if multiple project versions exist, remove content from models of each
    for pUUID in projectUUID:
        headers = {
            'Origin': host,
            'Authorization': f'Bearer {authToken}'}
        requestUrl = f'{host}/modelRepository/projects/{pUUID}/models?limit=100000'
        modelListGET = requests.get(requestUrl, headers=headers)
        modelListUUID = [x['id'] for x in modelListGET.json()['items']]
        for mUUID in modelListUUID:
            contentsUUID = getModelContents(host, authToken, mUUID)
            for cUUID in contentsUUID:
                code = deleteModelContents(host, authToken, mUUID, cUUID)
                if code.status_code == 204:
                    deleted += 1
    print(f'Deleted {str(deleted)} old files.')
    
# 3. All Content
# Delete content from all models in the common model repository.
elif removeType == 2:
    headers = {
        'Origin': host,
        'Authorization': f'Bearer {authToken}'}
    requestUrl = f'{host}/modelRepository/models?limit=100000'
    modelListGET = requests.get(requestUrl, headers=headers)
    modelListUUID = [x['id'] for x in modelListGET.json()['items']]
    for mUUID in tqdm(modelListUUID):
        contentsUUID = getModelContents(host, authToken, mUUID)
        for cUUID in contentsUUID:
            code = deleteModelContents(host, authToken, mUUID, cUUID)
            if code.status_code == 204:
                deleted += 1
    print(f'Deleted {str(deleted)} old files.')